In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

from pydmd import DMD
from pydmd.bopdmd import BOPDMD
from pydmd.plotter import plot_eigs
from pydmd.plotter import plot_summary
from pydmd.preprocessing.hankel import hankel_preprocessing

In [17]:
data = pd.read_csv('cdii.csv').drop(columns=['Unnamed: 0'])
data = deepcopy()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,2.067382,1.642595,1.303652,0.977397,0.891850,1.425976,1.250903,-0.136501,-0.769224,-1.030059,...,-0.933323,-0.846823,-0.469451,-0.036135,-0.014434,-0.241267,-0.554631,-0.683556,-0.238696,0.749885
1,2.612158,2.096693,1.436936,0.618676,-0.257318,-0.103003,0.071211,-0.336257,-0.915414,-0.744088,...,-1.558534,-0.787472,-0.257059,-0.228000,-0.056893,0.741745,1.082535,0.195655,-0.338819,-0.876988
2,2.333325,2.065002,1.665624,1.083547,0.735630,0.757885,0.635212,0.099607,-0.593399,-0.329325,...,-1.140988,-0.431763,-0.267953,-0.672272,-1.094901,-0.913872,-0.468089,-0.246102,0.079261,0.484752
3,1.929269,1.351132,0.615032,-0.650287,-0.905013,-1.264655,-1.776578,-1.367146,-0.948003,-1.053146,...,0.280304,1.525995,1.299182,0.284387,0.018673,0.339701,0.529637,0.588102,0.686134,0.861211
4,2.519172,1.937994,1.506739,1.129858,0.661267,0.941310,0.813749,0.165015,-0.355403,-0.535128,...,-0.899427,-0.700465,-0.703959,-0.880696,-0.609031,-0.336267,0.035596,-0.120175,-0.146656,-0.191950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,-1.732195,-1.028908,-0.375538,0.225858,0.718664,1.115790,1.380978,1.423856,1.235375,1.060843,...,0.318292,-0.099568,-0.352151,-0.555564,-0.635196,-1.112803,-1.284705,-1.271607,-1.243781,-1.143591
1803,2.264197,2.042080,1.757375,1.384045,1.123101,0.709674,0.271487,-0.226829,-0.469434,-0.696713,...,-0.872896,-0.641909,-0.397238,-0.237494,-0.300296,-0.278077,-0.253210,-0.199953,-0.180983,-0.195091
1804,2.059556,1.870958,1.681195,1.528948,0.840263,0.810004,0.081129,0.216045,-0.334687,-0.758491,...,-1.072478,-1.044906,-0.725137,-0.269770,0.013625,0.067548,0.188248,0.232729,-0.044633,-0.590252
1805,1.612925,1.575641,1.430484,1.262169,1.020239,1.014467,0.921732,0.617748,0.689684,0.563223,...,-0.782361,-1.048684,-1.092242,-0.977618,-1.054419,-1.495635,-1.373327,-1.029497,-0.696502,-0.326975


In [40]:
for i in np.random.choice(1807, size=10):
    print("Mean: ", np.mean(data.values[i,:]), " Std: ", np.std(data.values[i,:]))

Mean:  0.0  Std:  1.0
Mean:  -9.251858538542972e-17  Std:  1.0
Mean:  0.0  Std:  1.0
Mean:  7.401486830834377e-17  Std:  1.0000000000000002
Mean:  -1.1102230246251565e-16  Std:  0.9999999999999999
Mean:  1.3877787807814457e-16  Std:  0.9999999999999999
Mean:  1.850371707708594e-17  Std:  0.9999999999999998
Mean:  -7.864079757761526e-17  Std:  1.0
Mean:  1.295260195396016e-16  Std:  1.0
Mean:  1.850371707708594e-17  Std:  1.0
